#**What is customer churn?**

Customers churn when they terminate services with a company. In the context of Banking, customers churn when they stop using the services provided by the bank. Reasons for churn:


*   They simply no longer use their services
*   They go to Bank's competitors


**Part 1 - Data Preprocessing**

In [1]:
#!pip install pyforest
#from pyforest import *

In [2]:
# Importing needed libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import mannwhitneyu

from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

**Setting up plotly function**

In [3]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
  import plotly.graph_objs as go
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=True)

In [4]:
# Importing the dataset
dataset = pd.read_csv('../input/churn-predictions-personal/Churn_Predictions.csv')

In [5]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


We define some functions which will perform the crux of the Hypothesis testing in this notebook

In [6]:
def is_normal(group):
    """Check if input ndarray is normally distributed using the Shapiro Wilk Test"""
    wval, pval = stats.shapiro(group)
    if len(group) > 5000: # W-stat is more accurate for larger sample sizes
        if wval < 0.05: return False
    if pval < 0.05: return False
    return True

def has_eq_var(group_1, group_2):
    """Check the property of homoscendacity (equal variance) using the Levene test"""
    _, pval = stats.levene(group_1, group_2) 
    if pval < 0.05: return False
    return True

def stat_test(group_1, group_2):
    """Between 2 groups, if the conditions of normality + homoscendacity hold, perform 
    T-Test, otherwise perform the Mann Whiteny U Test."""
    if is_normal(group_1) and is_normal(group_2):
        if has_eq_var(group_1, group_2): 
            print("Performing T-Test ...")
            return stats.ttest_ind(group_1, group_2)
        print("Performing T-Test with unequal variance ...")
        return stats.ttest_ind(group_1, group_2, equal_var=False)
    print("Performing U-Test ...")
    return mannwhitneyu(group_1, group_2)

**1. Active Vs Churned Stats**

In [7]:
active_users = dataset[dataset['Exited']==0]
churned_users = dataset[dataset['Exited']==1]

active_male_users = active_users[active_users['Gender']=='Male']
churned_male_users = churned_users[churned_users['Gender']=='Male']

active_female_users = active_users[active_users['Gender']=='Female']
churned_female_users = churned_users[churned_users['Gender']=='Female']

In [8]:
enable_plotly_in_cell()
import plotly.graph_objs as go
from plotly.offline import iplot

labels_active = ['Male','Female']
values_active = [
    active_users[active_users['Gender']=='Male'].shape[0],
    active_users[active_users['Gender']=='Female'].shape[0],
                ]

trace_active = go.Pie(labels=labels_active, values=values_active, hole = .45, 
                      domain={"x": [0, .48]}, hoverinfo='label+percent+value')

labels_churned = ['Male','Female']
values_churned = [
    churned_users[churned_users['Gender']=='Male'].shape[0],
    churned_users[churned_users['Gender']=='Female'].shape[0],
                ]

trace_churned = go.Pie(labels=labels_churned, values=values_churned, hole = .5,
                       domain={"x": [0.52, 1]}, hoverinfo='label+percent+value')

layout={
    'title': 'Gender proportion for Active and Churned users',
    'annotations': [
        {'text': 'Active Users',                
         "x": 0.145, # Position of Text
         "y": 0.5,
         'showarrow':False,
         'font':{'size': 18}
        }, 
        {'text': 'Churned Users',
         "x": 0.875,
        "y": 0.5,
        'showarrow':False,
        'font':{'size': 18}
        }
    ]
}

fig = go.Figure(dict(data=[trace_active, trace_churned], layout=layout))
iplot(fig)

**Observation:**

*   Females make up the largest proportion (56%) of those who have exited the bank. 

**Geographical Customers (Active Vs Churned)**

In [9]:
enable_plotly_in_cell()
import plotly.graph_objs as go
from plotly.offline import iplot

labels_active = ['France','Spain', 'Germany']
values_active = [
    active_users[active_users['Geography']=='France'].shape[0],
    active_users[active_users['Geography']=='Spain'].shape[0],
    active_users[active_users['Geography']=='Germany'].shape[0]
                ]

trace_active = go.Pie(labels=labels_active, values=values_active, hole = .45, 
                      domain={"x": [0, .48]}, hoverinfo='label+percent+value')

labels_churned = ['France','Spain', 'Germany']
values_churned = [
    churned_users[churned_users['Geography']=='France'].shape[0],
    churned_users[churned_users['Geography']=='Spain'].shape[0],
    churned_users[churned_users['Geography']=='Germany'].shape[0]
                ]

trace_churned = go.Pie(labels=labels_churned, values=values_churned, hole = .50,
                       domain={"x": [0.52, 1]}, hoverinfo='label+percent+value')

layout={
    'title': 'Geographical stats for Active and Churned users',
    'annotations': [
        {'text': 'Active Users',                
         "x": 0.14, # Position of Text
         "y": 0.5,
         'showarrow':False,
         'font':{'size': 18}
        }, 
        {'text': 'Churned Users',
         "x": 0.88,
        "y": 0.5,
        'showarrow':False,
        'font':{'size': 18}
        }
    ]
}

fig = go.Figure(dict(data=[trace_active, trace_churned], layout=layout))
iplot(fig)

**Observation:**


> Customers in Germany and France constitute the largest proportion (40%) of customers who exit the bank. 


**1.2 France Tenure stats (Active vs Churned)**

In [10]:
active_france_users = active_users[active_users['Geography']=='France']
churned_france_users = churned_users[churned_users['Geography']=='France']

enable_plotly_in_cell()
import plotly.graph_objs as go
from plotly.offline import iplot

group_1 = go.Box(
    x=active_france_users['Tenure'],
    name="Active")

group_2 = go.Box(
    x=churned_france_users['Tenure'],
    name="Churned")

info = [group_1, group_2]
title = "Lifetime of Active Vs Churned France Customers"
iplot({
            "data": info, 
            "layout": {
                "title": title
            }
     })

In [11]:
group_1 = active_france_users['Tenure']
group_2 = churned_france_users['Tenure']
stat_test(group_1, group_2)

Performing U-Test ...


MannwhitneyuResult(statistic=1699669.5, pvalue=0.46868482201524836)

**Observation:** 
    
The U-Test failed to reject the null hypothesis that the 2 groups(Active and Churned) France customers come from the same distribution. Hence, France customers have no significant effect on their tenure with us.

**1.3 Spain Tenure stats (Active vs Churned)**

In [12]:
active_spain_users = active_users[active_users['Geography']=='Spain']
churned_spain_users = churned_users[churned_users['Geography']=='Spain']

In [13]:
enable_plotly_in_cell()
#import plotly.graph_objs as go
#from plotly.offline import iplot

group_1=go.Box(
    x=active_spain_users['Tenure'], name="Active")
group_2 = go.Box(
    x=churned_spain_users['Tenure'],
    name="Churned")

info = [group_1, group_2]
title = "Lifetime of Active Vs Churned Spain Customers"
iplot({
            "data": info, 
            "layout": {
                "title": title
            }
     })

In [14]:
group_1 = active_spain_users['Tenure']
group_2 = churned_spain_users['Tenure']
stat_test(group_1, group_2)

Performing U-Test ...


MannwhitneyuResult(statistic=388087.5, pvalue=0.001939485369542081)

**Observation:**

Active Spain customers stayed slightly longer with us than Churned customers from same country.

**1.4 Germany Tenure stats (Active vs Churned)**

In [15]:
active_germany_users = active_users[active_users['Geography']=='Germany']
churned_germany_users = churned_users[churned_users['Geography']=='Germany']

In [16]:
enable_plotly_in_cell()
#import plotly.graph_objs as go
#from plotly.offline import iplot

group_1=go.Box(
    x=active_germany_users['Tenure'], name="Active")
group_2 = go.Box(
    x=churned_germany_users['Tenure'],
    name="Churned")

info = [group_1, group_2]
title = "Lifetime of Active Vs Churned Germany Customers"
iplot({
            "data": info, 
            "layout": {
                "title": title
            }
     })

In [17]:
group_1 = active_germany_users['Tenure']
group_2 = churned_germany_users['Tenure']
stat_test(group_1, group_2)

Performing U-Test ...


MannwhitneyuResult(statistic=688994.0, pvalue=0.4794680464042427)

**Observation:** 
    
The U-Test failed to reject the null hypothesis that the 2 groups(Active and Churned) Germany customers come from the same distribution. Hence, Germany customers have no significant effect on their tenure with us.

**1.5 Geographical Tenure stats for Churned customers**

In [18]:
enable_plotly_in_cell()
#import plotly.graph_objs as go
#from plotly.offline import iplot

group_1=go.Box(
    x=churned_germany_users['Tenure'], name="Germany")
group_2 = go.Box(
    x=churned_spain_users['Tenure'],
    name="Spain")
group_3 = go.Box(
    x=churned_france_users['Tenure'],
    name="France")

info = [group_1, group_2,group_3 ]
title = "Lifetime of Churned Geographical Customers"
iplot({
            "data": info, 
            "layout": {
                "title": title
            }
     })

**1.6 Male Tenure stats (Active vs Churned)**

In [19]:
enable_plotly_in_cell()
group_1=go.Box(x=active_male_users['Tenure'], name="Active")
group_2 = go.Box(
    x=churned_male_users['Tenure'],
    name="Churned")

info = [group_1, group_2]
title = "Lifetime of Active Vs Churned Male Customers"
iplot({
            "data": info, 
            "layout": {
                "title": title
            }
       })

In [20]:
group_1 = active_male_users['Tenure']
group_2 = churned_male_users['Tenure']
stat_test(group_1, group_2)

Performing U-Test ...


MannwhitneyuResult(statistic=1989853.5, pvalue=0.09166640235084317)

**1.7 Female Tenure stats (Active vs Churned)**

In [21]:
enable_plotly_in_cell()
group_1=go.Box(x=active_female_users['Tenure'], name="Active")
group_2 = go.Box(
    x=churned_female_users['Tenure'],
    name="Churned")

info = [group_1, group_2]
title = "Lifetime of Active Vs Churned female Customers"
iplot({
            "data": info, 
            "layout": {
                "title": title
            }
       })

In [22]:
group_1 = active_female_users['Tenure']
group_2 = churned_female_users['Tenure']
stat_test(group_1, group_2)

Performing U-Test ...


MannwhitneyuResult(statistic=1920709.5, pvalue=0.3197071731996433)

**Gender tenure observation:**

The tenure length of either male or female customers seems not to have significant effect on their churn with us.

**1.8 Number of Products stats(Active vs Churned)**

In [23]:
active_users_products = active_users.NumOfProducts.value_counts()
churned_users_products = churned_users.NumOfProducts.value_counts()

enable_plotly_in_cell()
products=['1_product', '2_products', '3_products', '4_products']
fig = go.Figure(data=[
    go.Bar(name='Active_customers', y=active_users_products, x=products),
    go.Bar(name='Churned_customers', y=churned_users_products, x=products)
])
iplot(fig)

**Observations:**

*   By providing one or two products tends to have significantly more Active users compared to their respective churns.
*   Three products offered to customers tends to have more customers who churn compared to those who're active for same number of products.
*   All customers who had four products tend to exit the bank.

**Conclusion:**
By maximizing on offering a maximum of two products per customers will help retain more customers

**1.9 Number of Cards stats(Active vs Churned)**

In [24]:
active_users_products = active_users.HasCrCard.value_counts()
churned_users_products = churned_users.HasCrCard.value_counts()

enable_plotly_in_cell()
cards=['One Card', 'Two Cards']
fig = go.Figure(data=[
    go.Bar(name='Active_customers', y=active_users_products, x=cards),
    go.Bar(name='Churned_customers', y=churned_users_products, x=cards)
])
iplot(fig)

**Observations:**

*   Having one card tend to have a higher number of Active customers compared to those having two cards.
*   As a business, we need to evaluate whether or not it is necessary to provide two cards to customers.

**Part 2 - Predictive Modeling using Machine Learning techniques.**

In [25]:
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [26]:
geography = pd.get_dummies(X["Geography"], drop_first = True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [27]:
# Concatenate the Data Frames
X=pd.concat([X,geography,gender],axis=1)

In [28]:
# Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [29]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [30]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**2.1 Random Forest Classifier**

In [31]:
rf = RandomForestClassifier(n_estimators= 100)
bc = BaggingClassifier(rf, n_estimators=10, max_samples=0.6, max_features=1.0, bootstrap=True, bootstrap_features=False)
bc.fit(X_train, y_train)
print('Bagging Score: %0.2f%% ' %(bc.score(X_test, y_test)*100))

Bagging Score: 86.85% 


**2.2 Logistic Regression Classifier**

In [32]:
lr = LogisticRegression(solver='lbfgs', random_state=25)
bc = BaggingClassifier(lr, n_estimators=10, max_samples=0.6, max_features=1.0, bootstrap=True, bootstrap_features=False)
rfit = bc.fit(X_train, y_train)
print('Bagging Score: %0.2f%% ' %(bc.score(X_test, y_test)*100))

Bagging Score: 81.10% 


**2.3 KNearest Neighbors Classifier**



> Finding the best value for K using GridSearchCV Hyperparameter tuning technique.



In [33]:
# Instantiating logistic regression classifier 
knn = KNeighborsClassifier() 

# Creating the hyperparameter grid 
k_range = list(range(1, 51))
param_grid = {'n_neighbors':k_range } 

# Instantiating the GridSearchCV object 
knn_cv = GridSearchCV(knn, param_grid, cv = 10) 

knn_cv.fit(X_train, y_train) 

# Print the tuned parameters and score 
print("Tuned knn Parameters: {}".format(knn_cv.best_params_)) 
print("Best score is {}".format(knn_cv.best_score_))

Tuned knn Parameters: {'n_neighbors': 19}
Best score is 0.8313749999999999


In [34]:
knn = KNeighborsClassifier(n_neighbors = 19)
bc = BaggingClassifier(knn, n_estimators=10, max_samples=0.6, max_features=1.0, bootstrap=True, bootstrap_features=False, random_state=25)
bc.fit(X_train, y_train)
print('Bagging Score: %0.2f%% ' %(bc.score(X_test, y_test)*100))

Bagging Score: 84.05% 


**Part 3 - Now let's make the ANN!**

In [35]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential 
from keras.layers import Dense,LeakyReLU,PReLU,ELU,Dropout, BatchNormalization
from keras.activations import relu, sigmoid

Using TensorFlow backend.


In [36]:
# Initialising the ANN
classifier = Sequential()

In [37]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units= 6, kernel_initializer = 'he_uniform',activation='relu',input_dim = 11))
classifier.add(Dropout(0.3))

In [38]:
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))
#classifier.add(Dropout(0.4))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))
#classifier.add(Dropout(0.2))

In [39]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [40]:
# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, epochs = 100)

Train on 5359 samples, validate on 2641 samples
Epoch 1/100
5359/5359 [==============================] - 4s 667us/step - loss: 0.5869 - accuracy: 0.7960 - val_loss: 0.5189 - val_accuracy: 0.7955
Epoch 2/100
5359/5359 [==============================] - 2s 417us/step - loss: 0.5213 - accuracy: 0.7962 - val_loss: 0.4903 - val_accuracy: 0.7955
Epoch 3/100
5359/5359 [==============================] - 2s 443us/step - loss: 0.4953 - accuracy: 0.7960 - val_loss: 0.4730 - val_accuracy: 0.7955
Epoch 4/100
5359/5359 [==============================] - 2s 414us/step - loss: 0.4768 - accuracy: 0.7964 - val_loss: 0.4599 - val_accuracy: 0.7967
Epoch 5/100
5359/5359 [==============================] - 2s 423us/step - loss: 0.4607 - accuracy: 0.7970 - val_loss: 0.4479 - val_accuracy: 0.7959
Epoch 6/100
5359/5359 [==============================] - 2s 397us/step - loss: 0.4512 - accuracy: 0.8003 - val_loss: 0.4425 - val_accuracy: 0.7944
Epoch 7/100
5359/5359 [==============================] - 2s 397us/step

5359/5359 [==============================] - 2s 413us/step - loss: 0.3939 - accuracy: 0.8332 - val_loss: 0.3810 - val_accuracy: 0.8368
Epoch 57/100
5359/5359 [==============================] - 2s 458us/step - loss: 0.3921 - accuracy: 0.8334 - val_loss: 0.3793 - val_accuracy: 0.8342
Epoch 58/100
5359/5359 [==============================] - 3s 500us/step - loss: 0.3845 - accuracy: 0.8362 - val_loss: 0.3778 - val_accuracy: 0.8349
Epoch 59/100
5359/5359 [==============================] - 3s 467us/step - loss: 0.3879 - accuracy: 0.8334 - val_loss: 0.3767 - val_accuracy: 0.8368
Epoch 60/100
5359/5359 [==============================] - 2s 465us/step - loss: 0.3888 - accuracy: 0.8322 - val_loss: 0.3783 - val_accuracy: 0.8357
Epoch 61/100
5359/5359 [==============================] - 2s 421us/step - loss: 0.3870 - accuracy: 0.8270 - val_loss: 0.3784 - val_accuracy: 0.8323
Epoch 62/100
5359/5359 [==============================] - 2s 416us/step - loss: 0.3845 - accuracy: 0.8349 - val_loss: 0.3756 

In [41]:
# list all data in history
print(model_history.history.keys())
# summarize history for accuracy
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy'])


KeyError: 'acc'

In [ ]:
# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

**Part 3 - Making the predictions and evaluating the model**

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
print('The score accuracy is: %0.2f%%' %(score*100))

**Conclusion:**

From the above Machine Learning classifiers and ANN, it's clear that Random Forest Classifier performed best with accuracy > 86%. This is good but can we use Machine Learning in this? Well a useful application i can come up with is building a churn predictor. That is, given customer information, predict how likely he/she is likely to churn in the next month. But...


**NB:** 
> Although i have performed ML and ANN on this dataset, the data provided is not enough to justify using predictive modeling. I would need customer behavior over time such as:
>>1.   Where were they before joining the bank?
>>2.   When did they join us?

> We could extract features and throw it into an ML or ANN model but since we only have one tuple per customer, building this classier isn't really helpful given this data.